# Semi-supervised Dataset Preparation

- Mục tiêu: tạo bộ dữ liệu giữ cả phần **chưa có nhãn AQI** (aqi_class = NaN) để dùng cho self-training/co-training.
- Đồng thời **giả lập thiếu nhãn trong TRAIN** (time-aware) để mini project có thể thử nhiều mức thiếu nhãn.

In [1]:
CLEANED_PATH = "data/processed/cleaned.parquet"
OUTPUT_SEMI_DATASET_PATH = "data/processed/dataset_for_semi.parquet"
CUTOFF = "2017-01-01"
LABEL_MISSING_FRACTION = 0.95
RANDOM_STATE = 42

In [5]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np

# 1. Fix đường dẫn để nhận diện thư mục src
PROJECT_ROOT = Path(".").resolve()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from src.semi_supervised_library import SemiDataConfig, mask_labels_time_aware

# 2. Cấu hình các tham số theo yêu cầu dự án
# Mốc 2017-01-01 để chia Train/Test đảm bảo chống leak dữ liệu 
CUTOFF = "2017-01-01" 
# Giả lập thiếu 80% nhãn để thực hiện học bán giám sát [cite: 8, 19]
LABEL_MISSING_FRACTION = 0.8 

# 3. Đọc dữ liệu sạch bạn vừa tạo xong ở bước trước
input_file = PROJECT_ROOT / "data" / "processed" / "cleaned_data.parquet"
df = pd.read_parquet(input_file)

# 4. Thực hiện ẩn nhãn (Masking) ngẫu nhiên có kiểm soát [cite: 48, 50]
cfg = SemiDataConfig(cutoff=CUTOFF, random_state=42)
df2 = mask_labels_time_aware(df, cfg, missing_fraction=float(LABEL_MISSING_FRACTION))

# 5. Lưu tập dữ liệu bán giám sát để dùng cho Self-training/Label Spreading
out_path = PROJECT_ROOT / "data" / "processed" / "semi_labeled_data.parquet"
out_path.parent.mkdir(parents=True, exist_ok=True)
df2.to_parquet(out_path, index=False)

print("--- Hoàn thành Bước 2 ---")
print(f"Đã lưu tại: {out_path}")
print(f"Tổng số dòng: {len(df2)}")
# Kiểm tra xem cột 'is_labeled' đã xuất hiện chưa (True là có nhãn, False là không nhãn) [cite: 44]
print(f"Tỷ lệ mẫu giữ lại nhãn: {df2['is_labeled'].mean():.2%}")

--- Hoàn thành Bước 2 ---
Đã lưu tại: D:\air_guard_miniproject\data\processed\semi_labeled_data.parquet
Tổng số dòng: 420768
Tỷ lệ mẫu giữ lại nhãn: 22.80%
